In [1]:
import warnings
warnings.filterwarnings('ignore')

**Python library for analysis and generation of poems in Russian**

https://github.com/IlyaGusev/rupo

In [2]:
from rupo.api import Engine

Using TensorFlow backend.


In [3]:
engine = Engine(language='ru')

In [4]:
engine.load('/Users/nata/Desktop/stress_ru_LSTM64_dropout0.2_acc99_wer8.h5', '/Users/nata/Desktop/dict/zaliznyak.txt')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


**Коллекция стихотворений**

In [5]:
import urllib.request
import random
import time
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

import re
from pprint import pprint

In [6]:
t = random.randint(0, 3)

In [7]:
main_page = 'http://www.100bestpoems.ru/index.php'

In [8]:
def get_page_html(page_url):
    '''Получение HTML-кода страницы'''
    
    try:
        with urllib.request.urlopen(page_url) as response:
            html = response.read().decode('utf-8')
            soup = BeautifulSoup(html, 'html.parser')
    
    except HTTPError as e:
        print('Error code: ', e.code)
        pass
    except URLError as e:
        print('Reason: ', e.reason)
        pass
    
    return soup

In [9]:
def get_data(soups):
    '''Извлечение метаинформации и текста стихотворений'''
    
    titles, authors, dates, labels, languages, texts = [], [], [], [], [], []
    
    for soup in soups:
    
        title = soup.find('h1', attrs={'itemprop':'name'})
        if title:
            p_title = title.text.strip()
        else:
            p_title = None

        titles.append(p_title)

        author = soup.find('span', attrs={'itemprop':'name'})
        if author:
            p_author = author.text.strip()
        else:
            p_author = None

        authors.append(p_author)

        date = soup.find('td', attrs={'itemprop':'datePublished'})
        if date:
            p_date = date.text.strip()
        else:
            p_date = None

        dates.append(p_date)
        
        language = re.findall('<a href=".*?" title=".*?">(.*?)</a>', str(soup), re.DOTALL)
        if language:
            p_language = language[4]
        else:
            p_language = None

        languages.append(p_language)

        main_text = soup.find('span', attrs={'itemprop':'citation'})
        if main_text:
            p_text = main_text.text.strip().replace('\r', '\n').replace('\t', '')
        else:
            p_text = None

        texts.append(p_text)
    
    return titles, authors, dates, languages, texts

In [10]:
def item_info(href):
    '''Извлечение id страницы стихотворения (/item_info.php?id=348)'''
    
    return href and re.compile('item_info').search(href)

In [11]:
main_page_soup = get_page_html(main_page)
ids = main_page_soup.find_all(href=item_info)

In [12]:
def get_ids(ids):
    '''Извлечение id страницы стихотворения (/item_info.php?id=348)'''
    
    endings = []
    
    for element in ids:
        endings.append(element.get('href'))
    
    return endings

In [13]:
def get_poems_pages(endings):
    '''Получение HTML-кода страниц со стихотворениями'''
    poem_htmls = []
    
    urls = ['http://www.100bestpoems.ru/' + end for end in endings]
    time.sleep(t)

    for url in urls:
        poem_htmls.append(get_page_html(url))
        
    return poem_htmls, urls

In [14]:
endings = get_ids(ids)

In [15]:
poems_pages, urls = get_poems_pages(endings)

In [16]:
titles, authors, dates, languages, texts = get_data(poems_pages)

In [17]:
def write_file(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        for item in data:
            f.write("%s\n" % item)

In [18]:
write_file('/Users/nata/Desktop/titles.txt', titles)
write_file('/Users/nata/Desktop/authors.txt', authors)
write_file('/Users/nata/Desktop/dates.txt', dates)
write_file('/Users/nata/Desktop/languages.txt', languages)
write_file('/Users/nata/Desktop/texts.txt', texts)

**Анализ стихотворений**

In [19]:
def get_poetic_feet(poems):
    '''Определение стихотворного размера'''
    
    poetic_feet = []
    
    for poem in tqdm(poems):
        poetic_feet.append(engine.classify_metre(poem))
    
    return poetic_feet

In [20]:
feet = get_poetic_feet(texts)

A Jupyter Widget

In [21]:
import collections

In [22]:
counter = collections.Counter(feet)
print(counter)

Counter({'iambos': 45, 'choreios': 32, 'anapaistos': 19, 'dolnik3': 10, 'amphibrachys': 7, 'daktylos': 2})


In [23]:
write_file('/Users/nata/Desktop/feet.txt', feet)

In [24]:
import string
exclude = set(string.punctuation)

In [25]:
def clean_texts(texts):
    cleaned = []
    
    for text in texts:
        cleaned.append(''.join(ch for ch in text if ch not in exclude))
    
    return cleaned

In [26]:
cleaned = clean_texts(texts)

In [27]:
strings = []
for text in cleaned:
    s = [s.lstrip().replace('x', 'х') for s in text.split('\n')]
    strings.append(s)

In [28]:
idxs = []
i = 0
for st in strings:
    lst = [i] * len(st)
    idxs.append(lst)
    i += 1

In [29]:
def get_stress(texts):
    '''Определение ударения'''
    strings = []
    stresses = []
    
    for text in texts:
        s = [s.lstrip().replace('x', 'х') for s in text.split('\n')]
        strings.append(s)

    for s in strings:
        s_result = []
        for i in s:
            result = []
            for word in i.split():
                try:
                    result.append(word[:engine.get_stresses(word)[0]].lower() + \
                                    word[engine.get_stresses(word)[0]:].capitalize())
                except:
                    result.append(word.lower())
            
            s_result.append(result)
        
        stresses.append([' '.join(s) for s in s_result])
        
    return stresses

In [30]:
stresses = get_stress(cleaned)

In [31]:
def get_syllables(texts):
    syllables = []
    
    for text in texts:
        s_result = []
        for t in text:
            result = []
            for word in t.split():
                if engine.get_word_syllables(word) != []:
                    result.append('-'.join(engine.get_word_syllables(word)))
                else:
                    result.append(word.lower())
            
            s_result.append(result)
    
        syllables.append([' '.join(s) for s in s_result])
                
    return syllables

In [32]:
syllables = get_syllables(stresses)

In [33]:
psyllables = []

for s in syllables:
    psyllables.append('\n'.join(s))

In [34]:
write_file('/Users/nata/Desktop/stresses.txt', stresses)
write_file('/Users/nata/Desktop/syllables.txt', syllables)

**База данных**

In [35]:
import psycopg2

In [37]:
conn = psycopg2.connect(dbname='babj18bjqeymb5bro19a',
                        user='usgtuwljjcyc3nhepmmb',
                        host='babj18bjqeymb5bro19a-postgresql.services.clever-cloud.com',
                        password='uRMYqQ8g5na5UTFWe55I')

cur = conn.cursor()

In [38]:
# cur.execute('DROP TABLE poems, full_analysis, string_analysis')

**Создание таблиц**

In [39]:
cur.execute('CREATE TABLE poems \
            (id serial PRIMARY KEY, poem text, author text, title text, original_language text, date text, foot text, url text \
);')

In [40]:
cur.execute('CREATE TABLE full_analysis \
            (poem text, analysis text, feet text \
);')

In [41]:
cur.execute('CREATE TABLE string_analysis \
            (id serial Not NULL, poem_string text, analysis_string text \
);')

**Заполнение таблиц**

In [42]:
for idx, text, author, title, language, date, foot, url in zip(range(len(texts)), texts, authors, titles, languages, dates, feet, urls):
    cur.execute('INSERT INTO poems VALUES (%s, %s, %s, %s, %s, %s, %s, %s)',
                [idx, text, author, title, language, date, foot, url])

In [43]:
for poem_text, analysis_text, foot in zip(texts, psyllables, feet):
    cur.execute('INSERT INTO full_analysis VALUES (%s, %s, %s)',
                [poem_text, analysis_text, foot])

In [44]:
for idx, poem_text, analysis_text in zip(idxs, strings, syllables):
    for i, p, a in zip(idx, poem_text, analysis_text):
        cur.execute('INSERT INTO string_analysis VALUES (%s, %s, %s)', [i, p, a])

In [45]:
conn.commit()

**DELETE**

In [46]:
def check(table, j):
    cur.execute('SELECT * FROM %s' % table)
    rows = cur.fetchall()
    check_rows = []
    for i in range(len(rows)):
        check_rows.append(rows[i][j].lower())
    
    return check_rows

In [47]:
def delete_poem(title):
    if title.lower() not in check('poems', j=3):
        print('Такого стихотворения нет в базе данных.')
        
    else:
        cur.execute("DELETE FROM full_analysis WHERE poem IN (SELECT poem FROM poems WHERE title = '%s')" % title)
        cur.execute("DELETE FROM string_analysis WHERE id IN (SELECT id FROM poems WHERE title = '%s')" % title)
        cur.execute("DELETE FROM poems WHERE title = '%s'" % title)
        conn.commit()
        titles.remove(title)
        print(title + ' was deleted.')

In [48]:
delete_poem('VJHbjhd')

Такого стихотворения нет в базе данных.


In [ ]:
delete_poem('Отговорила роща золотая')

**SELECT**

In [49]:
def show_poems_by_author(name):
    '''Список стихотворений конкретного поэта'''
    
    if not any(name in t for t in authors):
        print('Такого поэта нет в базе данных.')
    
    try:
        names = [s for s in authors if name in s]

        query = "SELECT title FROM poems WHERE author = '%s'" % (names[0])
        cur.execute(query)
        poems = cur.fetchall()

        print(names[0] + ':\n')
        for poem in poems:
            pprint(''.join(poem))
    
    except:
        pass

In [50]:
show_poems_by_author('Блок')

Александр Блок:

'Ночь, улица, фонарь, аптека...'
'Незнакомка'
'Девушка пела в церковном хоре'


In [51]:
def show_poems_by_foot(foot):
    '''Список стихотворений конкретного строя'''
    
    if not any(foot in t for t in feet):
        print('Стихотворений такого строя нет в базе данных.')
    
    else:
        query = "SELECT title FROM poems WHERE foot LIKE '%s'" % (foot)
        cur.execute(query)
        poems = cur.fetchall()

        print(foot + ':\n')
        for poem in poems:
            pprint(''.join(poem))

In [52]:
show_poems_by_foot('anapaistos')

anapaistos:

'Мне осталась одна забава'
'Пой же, пой. На проклятой гитаре'
'Баллада о детстве'
'Кони привередливые'
'Он не вернулся из боя'
'Баллада о борьбе'
'Райские яблоки'
'Охота на волков'
'Вот и лето прошло'
'Я убит подо Ржевом'
'Да! Теперь решено. Без возврата...'
'Сукин сын (Снова выплыли годы из мрака...)'
'Ни о чем не жалейте'
'Заметался пожар голубой'
'Снова пьют здесь, дерутся и плачут...'
'Подражание песне'
'Моя жизнь'
'Баллада о борьбе'
'Память'


In [53]:
def show_details(title):
    '''Информация о стихотворении по его названию'''
    
    if not any(title in t for t in titles):
        print('Такого стихотворения нет в базе данных.')
    
    try:
        names = [s for s in titles if title in s]
        query = "SELECT author, title, original_language, date, foot, url FROM poems WHERE title = '%s'" % (names[0])
        cur.execute(query)
        rows = cur.fetchall()

        headers = ['Author', 'Title', 'Original language', 'Year', 'Foot', 'URL']
        print('{:<20s} {:<30s} {:<20s} {:<10s} {:<15s} {:<10s}'.format(*headers))
        for list_ in rows:
            print('{:<20s} {:<30s} {:<20s} {:<10s} {:<15s} {:<10s}'.format(*list_))
    
    except:
        pass

In [54]:
show_details('Ешь ананасы, рябчиков жуй...')

Author               Title                          Original language    Year       Foot            URL       
Владимир Маяковский  Ешь ананасы, рябчиков жуй...   Русский              1917       daktylos        http://www.100bestpoems.ru/item_info.php?id=8094


In [55]:
show_details('Зима')

Author               Title                          Original language    Year       Foot            URL       
Фёдор Тютчев         Зима недаром злится…           Русский              1836       iambos          http://www.100bestpoems.ru/item_info.php?id=5463


In [86]:
def show_analysis(title):
    '''Данные анализа стихотворения по его названию'''
    
    if not any(title in t for t in titles):
        print('Такого стихотворения нет в базе данных.')
    
    try:
        names = [s for s in titles if title in s]
        query = "SELECT poem_string, analysis_string, poems.author, poems.date, poems.foot \
                    FROM string_analysis \
                    FULL OUTER JOIN poems ON poems.id = string_analysis.id \
                    WHERE poems.title = '%s'" % (names[0])
        
        cur.execute(query)
        rows = cur.fetchall()
        print('Title: ' + title)
        print('Author: ' + rows[0][2])
        print('Year: ' + rows[0][3])
        print('Foot: ' + rows[0][-1] + '\n')

        headers = ['Poem text', 'Poem analysis']
        print('{:<60s} {:<60s}'.format(*headers))
        for list_ in rows:
            print('{:<60s} {:<60s}'.format(*list_))
            
    except:
        pass

In [87]:
show_analysis('Зима')

Title: Зима
Author: Фёдор Тютчев
Year: 1836
Foot: iambos

Poem text                                                    Poem analysis                                               
И снегу захватя                                              И снЕ-гу за-хва-тЯ                                          
И пуще лишь шумит                                            И пУ-ще лИшь шу-мИт                                         
Взбесилась ведьма злая                                       взбе-сИ-лась вЕ-дьма злА-я                                  
Зима недаром злится                                          зИ-ма не-дА-ром злИ-тся                                     
Прошла её пора —                                             прО-шла е-Ё пО-ра —                                         
Весна в окно стучится                                        ве-снА в о-кнО сту-чИ-тся                                   
И гонит со двора                                             И гО-нит сО дво-рА         

In [77]:
show_analysis('Новый год')

Новый год
[('Новогодние игрушки, свечи и хлопушки', 'н', 'Имя Фамилия', 'Год', 'choreios')]
Title: Новый год
Author: Имя Фамилия
Year: Год
Foot: choreios

Poem text                                                    Poem analysis                                               
Новогодние игрушки, свечи и хлопушки                         н                                                           


**INSERT**

In [58]:
def user_stresses(user_text):
    stresses = []
    for string in user_text.split('\n'):
        user_res = []
        for word in string.split():
            try:
                user_res.append(word[:engine.get_stresses(word)[0]].lower() + \
                            word[engine.get_stresses(word)[0]:].capitalize())

            except:
                user_res.append(word.lower())

        stresses.append(' '.join(user_res))

    return stresses

In [59]:
def user_syllables(user_text):
    syllables = []
    for string in user_stresses(user_text):
        user_res = []
        for word in string.split():
            if engine.get_word_syllables(word) != []:
                user_res.append('-'.join(engine.get_word_syllables(word)))
            else:
                user_res.append(word.lower())

        syllables.append(' '.join(user_res))
        
    
    
    return '\n'.join(syllables)

In [83]:
def insert_data(poem, author, title, year, language):    
    foot = engine.classify_metre(poem)
    
    texts.append(poem)
    
    cur.execute('INSERT INTO poems VALUES (%s, %s, %s, %s, %s, %s, %s)', 
                    (len(texts), poem, author, title, language, year, foot))
    
    lst = []
    i = len(poem.split('\n'))
    lst = [len(texts)] * i
    
    for l, p in zip(lst, poem.split('\n')):
        cur.execute('INSERT INTO string_analysis VALUES (%s, %s, %s)', 
                            (l, p, user_syllables(poem)))
    
    cur.execute('INSERT INTO full_analysis VALUES (%s, %s, %s)', 
                            (poem, user_syllables(poem), foot))
    
    conn.commit()
    
    titles.append(title)
    authors.append(authors)
    dates.append(year)
    languages.append(language)
    feet.append(foot)
    
    print(poem + ', ' + author + ', ' + title + ', ' + language + ', ' + year + ', ' + foot + ' were added.')

**Usage examples**

In [88]:
def user_input():
    # Извлечение различной информации
    user = input('Чтобы узнать, какие стихотворения уже содержатся в базе данных, введите 1:' + '\n')
    
    if user == '' or user != '1':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз!' + '\n')
    
    if user == '1':
        print('\n' + 'Названия стихотворений, содержащихся в базе данных: ')
        cur.execute("SELECT title FROM poems")
        
        for i in cur.fetchall():
            print('\n'.join(i))
        
    user = input('\n' + 'Чтобы узнать, какие авторы уже содержатся в базе данных, введите 2:' + '\n')
    
    if user == '' or user != '2':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз!' + '\n')
    
    if user == '2':
        print('\n' + 'Авторы, содержащиеся в базе данных: ')
        cur.execute("SELECT author FROM poems")
        authors = cur.fetchall()
        
        for i in set(authors):
            print('\n'.join(i))
    
    user = input('\n' + 'Чтобы узнать, какие стихотворения конкретного автора содержатся в базе данных, введите 3:' + '\n')
    
    if user == '' or user != '3':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')

    if user == '3':
        name = input('Введите автора:' + '\n')
        show_poems_by_author(name)
    
    user = input('\n' + 'Чтобы узнать, какие стихотворные размеры содержатся в базе данных, введите 4:' + '\n')
    
    if user == '' or user != '4':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')

    if user == '4':
        print('\n' + 'Стихотворные размеры, содержащиеся в базе данных: ')
        cur.execute("SELECT foot FROM poems")
        feet = cur.fetchall()
        
        for i in set(feet):
            print('\n'.join(i))
    
    user = input('\n' + 'Чтобы узнать, какие из имеющихся стихотворений относятся к тому или иному строю, введите 5:' + '\n')
    
    if user == '' or user != '5':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')

    if user == '5':
        foot = input('Введите название строя:' + '\n')
        show_poems_by_foot(foot)
        
    user = input('\n' + 'Чтобы получить информацию о конкретном стихотворении, введите 6:' + '\n')
    
    if user == '' or user != '6':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')

    if user == '6':
        title = input('Введите название стихотворения:' + '\n')
        show_details(title)
        
    
    user = input('\n' + 'Чтобы получить информацию о полном разборе данного стихотворения, введите 7:' + '\n')
    
    if user == '' or user != '7':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')

    if user == '7':
        show_analysis(title)
        
        
    # Выполнить анализ стихотворения и внести его в базу данных
    user = input('Чтобы добавить новое стихотворение и его анализ в базу данных, введите 9:' + '\n')
    
    if user == '' or user != '9':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')
    
    if user == '9':
        user = input('\n' + 'Введите стихотворение или одну из его строчек:' + '\n')
        
        cur.execute("SELECT poem_string FROM string_analysis")
        poem_strings = cur.fetchall()
        
        cleaned = []
        cleaned.append(''.join(ch for ch in user if ch not in exclude))

        for c in cleaned:
            matching = [s for s in poem_strings if c in s]

            if len(matching) > 0:
                print('Такое стихотворение уже есть в базе данных' + '\n')
                query = "SELECT full_analysis.poem, full_analysis.analysis, poems.title, poems.author, poems.date, poems.foot \
                    FROM string_analysis \
                    FULL OUTER JOIN poems ON poems.id = string_analysis.id \
                    FULL OUTER JOIN full_analysis ON poems.poem = full_analysis.poem \
                    WHERE poem_string = '%s'" % (matching[0])
        
                cur.execute(query)
                rows = cur.fetchall()
                
                print('Title: ' + rows[0][2])
                print('Author: ' + rows[0][3])
                print('Year: ' + rows[0][4])
                print('Foot: ' + rows[0][-1] + '\n')
                print('Poem text: ' + rows[0][0] + '\n')
                print('Poem analysis: ' + rows[0][1])
                    
            else:
                author = input('\n' + 'Введите автора стихотворения:' + '\n')
                title = input('\n' + 'Введите название стихотворения' + '\n')
                year = input('\n' + 'Введите год публикации стихотворения' + '\n')
                language = input('\n' + 'Введите язык оригинала стихотворения' + '\n')
                insert_data(user, author, title, year, language)
                show_analysis(title)
                
                
    # Удалить стихотворение и все данные о нем из базы данных
    user = input('Чтобы удалить конкретное стихотворение из базы данных, введите 10:' + '\n')
    
    if user == '' or user != '10':
        user = input('Похоже, что вы ввели пустой или неверный запрос. Попробуйте еще раз:' + '\n')
    
    if user == '10':
        user = input('\n' + 'Введите название стихотворения, чтобы удалить его:' + '\n')
    
        if user in titles:
            delete_poem(user)

        else:
            user = input('К сожалению, такого стихотворения нет в базе данных. Попробуйте еще раз:' + '\n')
            
            if user in titles:
                delete_poem(user)
    conn.commit()

In [89]:
user_input()

Чтобы узнать, какие стихотворения уже содержатся в базе данных, введите 1:
1

Названия стихотворений, содержащихся в базе данных: 
Не жалею, не зову, не плачу
Письмо матери
Отговорила роща золотая
Мне осталась одна забава
Чёрный человек
Евгений Онегин
Парус
Пой же, пой. На проклятой гитаре
Пускай ты выпита другим
Анна Снегина
До свиданья, друг мой, до свиданья
Выткался на озере алый свет зари
Я обманывать себя не стану
Баллада о детстве
Бородино
Песнь о собаке
Письмо к женщине
Жизнь — обман с чарующей тоскою...
Кони привередливые
Я вас любил: любовь еще, быть может...
Враги сожгли родную хату
Священная война
Исповедь хулигана
Он не вернулся из боя
К *** (Я помню чудное мгновенье)
Серафимыч (Николаю Резанову посвящается)
Зимнее утро
Хотят ли русские войны?
Смерть поэта
Журавли
Купола
Клён ты мой опавший,клён заледенелый
Широка страна моя родная
Пророк
Баллада о борьбе
Зимняя дорога
Стихи о советском паспорте
Райские яблоки
Ты меня не любишь не жалеешь
Песнь о вещем Олеге
Охота на волков